In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
import os
import pathlib
import dotenv
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint


In [2]:
from dotenv import load_dotenv
load_dotenv()

openai_api_key     = os.getenv("OPENAI_API_KEY")
gemini_api_key     = os.getenv("GEMINI_API_KEY")
groq_api_key       = os.getenv("GROQ_API_KEY")
langchain_api_key  = os.getenv("LANGCHAIN_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")

In [3]:
#loading The PDF

loader = PyPDFLoader(r"C:\Users\ishaa\Downloads\sample_medical_report.pdf")
medical_report = loader.load()
print("medical report loaded")
print(f"loaded doc has : {len(medical_report)} Page")


medical report loaded
loaded doc has : 1 Page


In [4]:
#text Spiltter 
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap = 20,
    length_function = len, #To count the CharS
)
chunks = splitter.split_documents(medical_report)
print("Chunks created")
print(f"{len(chunks)} : length of chunks")      

Chunks created
7 : length of chunks


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'} 
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

print("Embedding model loaded From Huggingface")

C:\Users\ishaa\AppData\Local\Temp\ipykernel_16928\1603069342.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Embedding model loaded From Huggingface


In [6]:
#The embeddings and the vectpor Db part 
db = FAISS.from_documents(chunks,embeddings)
db.save_local("faiss_index")
current_directory = os.getcwd()
index_path = os.path.join(current_directory, "faiss_index")
print(f"The Embeddings have been made @ {index_path}")

The Embeddings have been made @ c:\Users\ishaa\OneDrive\Desktop\AgenticAi\Agentic-Ai\Udmy\Project-Synaptic\faiss_index


In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

print("Databade Loaded")

Databade Loaded


In [8]:
#Creating The context retriever 

retriever = db.as_retriever()
print(f"retriever created : \n {retriever}")

retriever created : 
 tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000207ED384050> search_kwargs={}


In [9]:
# Lets make The user choose whatever model he/she wantes to use: 
#function will be Createsd to let  User Choose between 
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAI
from langchain_perplexity import ChatPerplexity 
from langchain_ollama import ChatOllama

In [ ]:
user_choice = input("enter the model: Perplexity , Gemini , groq ")
user_choice.lower()
if user_choice == "perplexity":
    


SyntaxError: incomplete input (638118774.py, line 4)